# Production Data Loading

 1. Access Couchbase Database
 1. Load CSV files and convert into JSON structure 
 1. Store the JSON data in Couchbase 

## Access Couchbase Database

  * Created new Couchbase _bucket_ called _geotechnical_ using the Couchbase web browser interface

In [9]:
from couchbase.bucket import Bucket
cb = Bucket('couchbase://localhost/geotechnical')  # If no errors then the access works

## Load CSV Data

In [10]:
# modified JK 2017.06.22

import json

from GeopyUtils import CsvDataReader

##projects_cdr = CsvDataReader('project', 'data/NEM/projects.csv')
##projects = projects_cdr.read()

##borehole_cdr = CsvDataReader('borehole', 'data/NEM/boreholes.csv')
##boreholes = borehole_cdr.read()

# data folder moved to  
parameter_cdr = CsvDataReader('parameter', 'data/NT1/2018.04.14.LabData.R2 - NT1.DataOrganiser.Parameters.csv')
parameters = parameter_cdr.read()

print "Record counts: "
##print "  projects:   " + str(len(projects))
##print "  boreholes:  " + str(len(boreholes))
print "  parameters: " + str(len(parameters))
##print "TOTAL: "+str(len(projects)+len(boreholes)+len(parameters))
##print "\nExample JSON data: project[0]"
##print json.dumps(projects[0], indent=2)

##print "\nExample JSON data: borehole[0]"
##print json.dumps(boreholes[0], indent=2)

print "\nExample JSON data: parameters[180]"
print json.dumps(parameters[180], indent=2)

Record counts: 
  parameters: 294

Example JSON data: parameters[180]
{
  "sample_description": "Basic Design", 
  "measurement_convention": "E50", 
  "test_standard": "ASTM D7012 D", 
  "sample_code": "13", 
  "test_parameters": [
    {
      "units": "", 
      "description": "", 
      "value": ""
    }, 
    {
      "units": "", 
      "description": "", 
      "value": ""
    }
  ], 
  "laboratory": "", 
  "test_comment": "", 
  "sample_orientation": "80", 
  "project_code": "NT1.r2", 
  "type": "parameter", 
  "laboratory_test_code": "61", 
  "test_type": "E measured during uniaxial compression", 
  "value_scale": "Laboratory", 
  "id": "parameter-20180414-143837-94966723", 
  "parameter": "E", 
  "revision": "r2", 
  "lithology": "Siltstone", 
  "sample_depth_top_m": "82.91", 
  "input_id": "181", 
  "value_likelihood": "Data point", 
  "data_source": "Laboratory", 
  "borehole_code": "NT1-5", 
  "value": "1413.00", 
  "test_date": "", 
  "value_unit": "MPa"
}


# Store in Couchbase

 * Only run this once!
 * Then comment out the *cb.insert* statement

In [11]:
from couchbase.bucket import Bucket
cb = Bucket('couchbase://localhost/geotechnical')

def load_records(records):
    for record in records:
        id = record['id']
        cb.upsert(id,record)  # DISABLED!!!
        print "record: "+id+" ..inserted"

##print "\nLoading: projects"
##load_records(projects)

##print "\nLoading: boreholes"
##load_records(boreholes)

print "\nLoading: parameters"
load_records(parameters)


Loading: parameters
record: parameter-20180414-143837-77271773 ..inserted
record: parameter-20180414-143837-64736494 ..inserted
record: parameter-20180414-143837-05075429 ..inserted
record: parameter-20180414-143837-95129819 ..inserted
record: parameter-20180414-143837-59014018 ..inserted
record: parameter-20180414-143837-84095427 ..inserted
record: parameter-20180414-143837-61154069 ..inserted
record: parameter-20180414-143837-39966408 ..inserted
record: parameter-20180414-143837-94862357 ..inserted
record: parameter-20180414-143837-15421394 ..inserted
record: parameter-20180414-143837-40925823 ..inserted
record: parameter-20180414-143837-48693354 ..inserted
record: parameter-20180414-143837-63489389 ..inserted
record: parameter-20180414-143837-99933489 ..inserted
record: parameter-20180414-143837-96181466 ..inserted
record: parameter-20180414-143837-58766155 ..inserted
record: parameter-20180414-143837-70396336 ..inserted
record: parameter-20180414-143837-27230958 ..inserted
record:

record: parameter-20180414-143837-10804899 ..inserted
record: parameter-20180414-143837-81523806 ..inserted
record: parameter-20180414-143837-93226971 ..inserted
record: parameter-20180414-143837-78863351 ..inserted
record: parameter-20180414-143837-10018374 ..inserted
record: parameter-20180414-143837-45338784 ..inserted
record: parameter-20180414-143837-91579060 ..inserted
record: parameter-20180414-143837-89161759 ..inserted
record: parameter-20180414-143837-26070200 ..inserted
record: parameter-20180414-143837-14862642 ..inserted
record: parameter-20180414-143837-58657230 ..inserted
record: parameter-20180414-143837-68973100 ..inserted
record: parameter-20180414-143837-06746418 ..inserted
record: parameter-20180414-143837-76398377 ..inserted
record: parameter-20180414-143837-35808520 ..inserted
record: parameter-20180414-143837-93952961 ..inserted
record: parameter-20180414-143837-54182063 ..inserted
record: parameter-20180414-143837-58502542 ..inserted
record: parameter-20180414-1

# Read Some Data

## Create Default Index

 * Required for querying. If you don't do this you get an error when you run queries
 * This should only be run once. Note yet sure how to ensure that happend
 * For now, run it once and then comment out the *cb.n1ql_query* statement

In [12]:
from couchbase.bucket import Bucket

# create a primary index on geotechnical bucket in couchbase
cb = Bucket('couchbase://localhost/geotechnical')
##cb.n1ql_query('CREATE PRIMARY INDEX ON geotechnical').execute()  # TODO this should only be run one time!
# running above query gave HTTPError: ... http_status=500. Primry Index was however created (couchbase browser).

In [13]:
import json

# check new data stored
from DataOrganizer import CouchbaseSelector

# -- Open the database

db = CouchbaseSelector('localhost','geotechnical')

##fields = ['parameter', 'type', 'value', 'value_unit']
##where = {'project_code':'NT1', 'type': 'parameter', 'parameter': 'E','data_source': 'Laboratory'}
##db.getParameter(parameter_name, fields, where)

recs = []
for rec in db.select(None, None):
# comment out following line for checking data before uploading
# uncomment for uploading
##for rec in db.select(fields, where):
    print json.dumps(rec, indent=2)
    recs.append(rec)
    
print json.dumps(recs[0],indent=2)
print "\nTotal records: "+str(len(recs))

{
  "geotechnical": {
    "borehole_code": "NE1-05", 
    "collar_elevation_masl": "", 
    "y_project_coordinate": "1721741.8", 
    "x_project_coordinate": "712184.8", 
    "type": "borehole", 
    "id": "borehole-20170622-174516-05582985"
  }
}
{
  "geotechnical": {
    "borehole_code": "NE1-12", 
    "collar_elevation_masl": "", 
    "y_project_coordinate": "1719777.5", 
    "x_project_coordinate": "717089.3", 
    "type": "borehole", 
    "id": "borehole-20170622-174516-11300620"
  }
}
{
  "geotechnical": {
    "borehole_code": "NE1-08", 
    "collar_elevation_masl": "", 
    "y_project_coordinate": "1708099.92", 
    "x_project_coordinate": "716316.28", 
    "type": "borehole", 
    "id": "borehole-20170622-174516-23912291"
  }
}
{
  "geotechnical": {
    "borehole_code": "NE1-07", 
    "collar_elevation_masl": "", 
    "y_project_coordinate": "1722689", 
    "x_project_coordinate": "711228.7", 
    "type": "borehole", 
    "id": "borehole-20170622-174516-27653986"
  }
}
{
  "geo

{
  "geotechnical": {
    "value_scale": "", 
    "measurement_convention": "", 
    "test_standard": "", 
    "laboratory": "", 
    "test_parameters": [
      {
        "units": "MPa", 
        "description": "s_comp", 
        "value": "3.20"
      }, 
      {
        "units": "", 
        "description": "", 
        "value": ""
      }
    ], 
    "sample_code": "21", 
    "test_comment": "", 
    "sample_orientation": "", 
    "project_code": "NT1.r2", 
    "laboratory_test_code": "", 
    "test_type": "Plate Bearing", 
    "sample_description": "Basic Design", 
    "id": "parameter-20180414-143837-14862642", 
    "type": "parameter", 
    "revision": "r2", 
    "lithology": "Sandstone", 
    "sample_depth_top_m": "17.00", 
    "input_id": "265", 
    "value_likelihood": "", 
    "parameter": "E", 
    "data_source": "Field", 
    "borehole_code": "NT1-RB-ExpAdit", 
    "value": "6450.00", 
    "test_date": "", 
    "value_unit": "MPa"
  }
}
{
  "geotechnical": {
    "value_scale"